In [40]:
# import library
import pandas as pd
import numpy as np 
import matlab 
import matplotlib as plt
import os
import numpy as np


In [52]:
# read the file
# mortality
mortality_path_7 = r'data/MortIcd7.csv'
mortality_data_7 = pd.read_csv(mortality_path_7, low_memory=False)

mortality_path_8 = r'data/MortIcd8.csv'
mortality_data_8 = pd.read_csv(mortality_path_8, low_memory=False)

mortality_path_9 = r'data/Morticd9.csv'
mortality_data_9 = pd.read_csv(mortality_path_9, low_memory=False)

mortality_path_2002 = r'data/Morticd10_part1.csv'
mortality_data_2002 = pd.read_csv(mortality_path_2002, low_memory=False)

mortality_path_2007 = r'data/Morticd10_part2.csv'
mortality_data_2007 = pd.read_csv(mortality_path_2007, low_memory=False)

mortality_path_2012 = r'data/Morticd10_part3.csv'
mortality_data_2012 = pd.read_csv(mortality_path_2012, low_memory=False)

mortality_path_2016 = r'data/Morticd10_part4.csv'
mortality_data_2016 = pd.read_csv(mortality_path_2016, low_memory=False)

mdf = pd.concat([mortality_data_7, mortality_data_8, mortality_data_9, mortality_data_2002, mortality_data_2007, mortality_data_2012, mortality_data_2016], ignore_index=True)

# population
pop_path = r'data/pop.csv'
pdf = pd.read_csv(pop_path, low_memory=False)

# country code
country_code_path = r'data/country_codes.csv'
country_code = pd.read_csv(country_code_path, low_memory=False)
# creating the country code map 
country_code_map = {}
for index, rows in country_code.iterrows():
    country_code_map[rows['country']] = rows['name']

In [59]:
mdf['Year'].isnull().sum()

0

In [54]:
# working, deleted the not useful columns
undesired_mortality_columns = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4'] 
undesired_population_columns = ['Admin1', 'SubDiv', 'Sex', 'Frmat'] 
mdf = mdf.drop(columns = undesired_mortality_columns)
pdf = pdf.drop(columns = undesired_population_columns)

In [68]:
# needs to swap out the nah columns into averages 
not_used_country_list = []
for key in country_code_map:
    country_mdf = mdf[mdf['Country'] == key]
    if country_mdf.empty:
        not_used_country_list.append(key)
    else:
        min_year = int(country_mdf['Year'].min())
        max_year = int(country_mdf['Year'].max()) 
        for i in range (min_year, max_year + 1):
            year_mdf = country_mdf[country_mdf['Year'] == i]
            for c in year_mdf.columns:
                mdf = mdf.loc[(mdf['Country'] == key) & (mdf['Year'] == i)][c].fillna(year_mdf[c].mean()) 
        

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_28872\2782037953.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdf.loc[(mdf['Country'] == key) & (mdf['Year'] == i)][c] = mdf[c].fillna(year_mdf[c].mean())


KeyboardInterrupt: 

In [28]:
for key in country_code_map:
    country_mdf = mdf[mdf['Country'] == key]
    display(country_mdf)

In [ ]:
# looping through the code map to locate each different countries
# condense the mortality and pop rate 
undesired = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4']
modified_dict = {}
for i in mdf.columns.values:
    if i not in undesired:
       modified_dict[i] = []

modified_df = pd.DataFrame(modified_dict)
list_delete_keys = []
consider_population = True

for key in country_code_map: 
    temp_mortality_df = mdf[mdf['Country'] == key] # filter to the country
    temp_pop_data = pop_data[pop_data['Country'] == key]
    if temp_mortality_df.empty:
        list_delete_keys.append(key)
    else:
        for i in range (1980, 2003):
            temp_temp_mortality_df = temp_mortality_df.loc[temp_mortality_df['Year'] == i] # filter to the year 
            temp_temp_pop_data = temp_pop_data.loc[temp_pop_data['Year'] == i] 
            if temp_temp_mortality_df.empty:
                continue
            else:
                modified_df.loc[modified_df.shape[0]] = ['None'] * modified_df.shape[1]
                modified_df.loc[modified_df.shape[0]-1, ['Country']] = key
                modified_df.loc[modified_df.shape[0]-1, ['Year']] = i
                column_name = modified_df.columns.values 
                k = 1
                for c in column_name[2:]:
                    j = f'Pop{k}'
                    population = temp_temp_pop_data[j].sum()
                    if c in temp_temp_mortality_df.columns.values:
                        if consider_population:
                            if population == 0:
                                modified_df.loc[modified_df.shape[0]-1, [c]] = 0
                            else:
                                modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum() / population
                        else:    
                            modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum() 
                    k += 1
            
                # modified_df.index = modified_df.index + 1
                # modified_df = modified_df.sort_index()      

In [ ]:
# delete keys from dictionary
for key in list_delete_keys:
    del country_code_map[key]

In [ ]:
# building the mortality model 
country_mortality_model = {}
final_beginning_year = 9999
for key in country_code_map:
    # isolate country
    temp_modified_df = modified_df[modified_df['Country'] == key]
    # temp_modified_df.drop(columns=['Country'], inplace = True) 
    
    # finding alpha 
    last_year = temp_modified_df['Year'].max()
    
    if last_year < final_beginning_year:
        final_beginning_year = last_year
    
    first_year = temp_modified_df['Year'].min()
    duration = last_year - first_year + 1
    last_row = temp_modified_df[temp_modified_df['Year'] == last_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    first_row = temp_modified_df[temp_modified_df['Year'] == first_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    aphla = (last_row - first_row) / duration

    country_mortality_model[key] = {'a': aphla, 'd': duration}

In [ ]:
final_ending_year = 2050
current_year = 2000

mortality_pred_all = []
while current_year <= final_ending_year:
    
    mortality_pred_year = {}
    
    for key in country_code_map: 
        
        print(key)
        
        # isolate the data to the target data row \n",
        temp_modified_df = modified_df[(modified_df['Country'] == key)]  
        # check if there is real data for this year for this country\n",
        if current_year <= 2002:
            df = temp_modified_df[temp_modified_df['Year'] == current_year + 1]
            if not df.empty:
                df = df.loc[:, ~df.columns.isin(['Country', 'Year'])].values.astype('float')
                mortality_pred_year[key] = df 
                continue
 
        # clean the data for the row
        temp_modified_df = temp_modified_df[temp_modified_df['Year'] == current_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
        # find the aphla, b and k value for the model for future \n",
        aphla = country_mortality_model[key]['a']
        duration = country_mortality_model[key]['d']
        m_hat = (temp_modified_df - aphla)
        s, u, v = np.linalg.svd(m_hat)
        scalar = s[0]
        k = scalar * u[0]
        b = v[0] / scalar 
  
        # compute the next years result",
        change = np.exp(aphla + b * k)
        display(temp_modified_df.shape)
        display(change.shape)
        next_year = temp_mortality_df + change
        mortality_pred_year[key] = next_year
        
        # update parameters
        duration += 1
        aphla = (aphla * duration + change) / duration
        country_mortality_model[key]['a'] = aphla
        country_mortality_model[key]['d'] = duration
    
    current_year += 1
    mortality_pred_all.append(mortality_pred_year)
        
        


In [ ]:
key = 1125
temp_modified_df = modified_df[(modified_df['Country'] == key)]  
# clean the data for the row
temp_modified_df = temp_modified_df[temp_modified_df['Year'] == current_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
# find the aphla, b and k value for the model for future \n",
aphla = country_mortality_model[key]['a']
duration = country_mortality_model[key]['d']
m_hat = (temp_modified_df - aphla)
s, u, v = np.linalg.svd(m_hat)
scalar = s[0]
k = scalar * u[0]
b = v[0] / scalar 

display(aphla)
display(k, b)
change = np.exp(aphla + b * k)
display(temp_modified_df)
display(change)
display(temp_modified_df.shape)
next_year = temp_mortality_df + change
mortality_pred_year[key] = next_year